## AES192 Encryption and Decryption

In [ ]:
import numpy as np
from matplotlib import image
from matplotlib import pyplot
import matplotlib.pyplot as plt
np.set_printoptions(formatter={'int':hex})
import IPython
from siha import *
import pad
from IPython.display import Image
filename = 'images/andromeda.png'
efilename = 'images/andromeda192.enc.png'
dfilename = 'images/andromeda192.dec.png'

In [ ]:
Image(filename=filename)

# Encryption

In [ ]:
BUFFSIZE = 0x1000000
databuffOrg = np.fromfile(filename, dtype=np.uint8)[:BUFFSIZE]
databuff, padcount = pad.addpad(databuffOrg)
databuff = np.frombuffer(databuff.tobytes(), dtype=np.uint32)
size = databuff.shape[0]

keybuff = np.array([
    0x03020100, 0x07060504, 0x0b0a0908, 0x0f0e0d0c,
	0x13121110, 0x17161514, 0x00000001, 0x00000000], dtype=np.uint32)

unkeybuff = np.array([
    0x03020100, 0x07060504, 0x0b0a0908, 0x0f0e0d0c,
	0x13121110, 0x17161514, 0x00000000, 0x00000000], dtype=np.uint32)

tid_config = 1
tid_run = 0
offset_key = 0
offset_input = 64
offset_result = 64
size_config = len(keybuff)

accel = AcapdAccel("AES192", 0)
accel.loadData(keybuff, offset_key)
accel.loadData(databuff, offset_input)
accel.startAccel()
accel.config(offset_key, size_config, tid_config)
accel.run(offset_input, size, tid_run, offset_result)
outbuff = accel.retrieve(offset_result, size)
accel.close()
outbuff.tofile(efilename)
img = np.frombuffer(outbuff.tobytes(), dtype=np.uint8)[-614* 1024* 3:].reshape((614, 1024, 3))
pyplot.imshow(img)
pyplot.show();

# Decryption

In [ ]:
BUFFSIZE = 0x1000000
databuff = np.fromfile(efilename, dtype=np.uint32)
accel = AcapdAccel("AES192", 0)
accel.loadData(unkeybuff, offset_key)
accel.loadData(databuff, offset_input)
accel.startAccel()
accel.config(offset_key, size_config, tid_config)
accel.run(offset_input, size ,tid_run, offset_result)
outbuff = accel.retrieve(offset_result, size)
accel.close()
outbuff = np.frombuffer(outbuff.tobytes(), dtype=np.uint8)
outbuff = outbuff[:-padcount]
outbuff.tofile(dfilename)
img = image.imread(dfilename)
pyplot.imshow(img)
pyplot.show();

In [ ]:
Image(filename=dfilename)